# SK Car Sales Descriptions in English and Spanish

- Get a list of cars and for each car get a sales descriptions in English and translate it into Spanish.

## Load the required .NET packages and supporting classes

In [1]:
#r "nuget: Microsoft.SemanticKernel, 0.18.230725.3-preview"
#r "nuget: Microsoft.Extensions.Logging.Console, 7.0.0"
#r "nuget: Microsoft.EntityFrameworkCore.Sqlite, 7.0.9"
#r "nuget: dotenv.net, 3.1.2"

using System.ComponentModel;
using System.Net.Http;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Skills.Core;
using Microsoft.SemanticKernel.SkillDefinition;
using Microsoft.EntityFrameworkCore;
using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#!import Utils/ConsoleLogger.cs
#!import Models/Models.cs

// Json Options
readonly JsonSerializerOptions s_jsonOptions = new() { WriteIndented = true };

Installed Packages dotenv.net, 3.1.2 Microsoft.EntityFrameworkCore.Sqlite, 7.0.9 Microsoft.Extensions.Logging.Console, 7.0.0 Microsoft.SemanticKernel, 0.18.230725.3-preview

In [2]:
class CarsDbContext : DbContext
{
    public DbSet<CarEF> Users { get; set; }

    public CarsDbContext()
    {
    }

    public CarsDbContext(DbContextOptions<CarsDbContext> options) : base(options)
    {
    }
}

public class CarEF
{
    public string VIN { get; set; }
    public string Make { get; set; }
    public string Model { get; set; }
    public int Year { get; set; }
    public string Package {get;set;}
    public string Motor {get;set;}
    public double Price {get;set;}
}

## Load the OpenAI variables from environment variables or an .env file

In [3]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_KEY");
Console.WriteLine($"Using deployment: {deploymentName} at: {endpoint} with key {apiKey.Substring(0, 5)}...");

Using deployment: davinci at: https://eualemor.openai.azure.com/ with key bd987...


## Mock a database call to get the a list of cars

In [4]:
var mockCarData = new List<Car>{
    new Car("123","Ford", "Explorer", 2015, "Silver", "V6", "Platinum", 10000, 16500),
    new Car("456","Ford", "Mustang", 2018, "Blue", "V8", "Sports", 10000, 250000),
    new Car("789","Ford", "Escape", 2020, "Red", "V6","Special", 3000, 15000)
  };
  
List<Car> MockDBCall() {  
  return mockCarData;
}

## Function to get a short car description from the car properties

In [5]:
string GetCarDescription(Car car) {
  return $"{car.Year} {car.Make} {car.Model} {car.Color} {car.Motor} {car.Package} with {car.milage} miles for ${car.price}";
}

## Create an instance of a kernel

In [6]:
var kernel = new KernelBuilder()
    .WithLogger(ConsoleLogger.Log)
    .WithAzureTextCompletionService(deploymentName,endpoint, apiKey)
    .Build();

## Create the sales definition and translation in-line functions and add them to the kernel

In [7]:
// Define a function to get a used car sales description
const string fnSalesDefinition = @"Get a used car sales description for the following: 
{{$input}}.";

var salesDescriptionFunc = kernel.CreateSemanticFunction(fnSalesDefinition, 
    maxTokens: 200, temperature: 0.5, topP: 1);

// Define a function to translate English to Spanish
const string fnESTranslatorDefinition = @"Translate the following English text to Spanish: 
{{$input}}.";

var esTranslatorFunc = kernel.CreateSemanticFunction(fnESTranslatorDefinition, 
maxTokens: 200, temperature: 0.5, topP: 1);

## Using the kernel and sk functions, get a car sales description and translate it into Spanish

In [8]:
// Mock making a call to a database to get a list of cars
var cars = MockDBCall();
var text = kernel.ImportSkill(new TextSkill());
var carDesc = GetCarDescription(cars[0]);

## Get a sales description, translate it and Uppercase it in a pipe

In [9]:

var pipeLineResult = await kernel.RunAsync(carDesc,
  salesDescriptionFunc,
  esTranslatorFunc,
  text["Uppercase"]);

Console.WriteLine(pipeLineResult);



¡ESTA ES UNA GRAN OFERTA PARA UN FORD EXPLORER SILVER V6 PLATINUM DEL 2015 CON SOLO 10.000 MILLAS! ESTÁ EN EXCELENTES CONDICIONES Y ESTÁ CARGADO DE CARACTERÍSTICAS COMO ASIENTOS DE CUERO, UN TECHO SOLAR ELÉCTRICO Y UNA CÁMARA TRASERA. EL MOTOR V6 PROPORCIONA MUCHA POTENCIA Y EL ACABADO PLATINUM LE DA LUJOSAS COMODIDADES COMO UN PANEL DE INSTRUMENTOS DIGITAL Y PEDALES AJUSTABLES ELÉCTRICAMENTE. ¡NO TE PIERDAS ESTA INCREÍBLE OPORTUNIDAD DE POSEER ESTE HERMOSO SUV POR SOLO $16.500!


## Get a sales description and a translation for all the cars in the mock database

In [10]:
// Run tasks asynchronously
// var task1 = kernel.RunAsync(carDesc,salesDescriptionFunc);
// var task2 = kernel.RunAsync(carDesc,salesDescriptionFunc);
// await Task.WhenAll(task1,task2);
// Console.WriteLine(task1.Result + " " + task2.Result);

var processedCars = new List<CarTranslation>();
foreach(var car in cars) {

  // Get a short description given the car properties
  var desc = GetCarDescription(car);
  Console.WriteLine($"Processing: {desc}");
  
  // Use the SK sales description function to get a sales for a car
  var result = await kernel.RunAsync(desc,salesDescriptionFunc);
  Console.WriteLine($"Sales Description: {result}");

  // Get the usage for the call
  Console.WriteLine(JsonSerializer.Serialize(result.ModelResults.LastOrDefault()?.GetOpenAITextResult()?.Usage, s_jsonOptions));
  
  // Use the SK translation function to translate the sales description into Spanish
  var es = await kernel.RunAsync(result.ToString(), esTranslatorFunc);
  Console.WriteLine($"Translation: {es}");
  
  // Add processed the EN and ES sales description
  processedCars.Add(new CarTranslation(car.VIN, result.ToString(), es.ToString()));
}

// Show the processed cars
processedCars

Processing: 2015 Ford Explorer Silver V6 Platinum with 10000 miles for $16500
Sales Description: 

This 2015 Ford Explorer Platinum is the perfect combination of luxury and power. It is finished in a sleek silver exterior that stands out from the crowd, and it is powered by a V6 engine that provides plenty of power. With only 10,000 miles, this Explorer is in excellent condition and ready for its next owner. Inside, you'll find a luxurious interior with all the bells and whistles you could want. Enjoy features like leather seating, a power moonroof, and a premium sound system. All of this is available for the great price of only $16,500. Don't miss your chance to own this beautiful Ford Explorer!
{
  "CompletionTokens": 131,
  "PromptTokens": 27,
  "TotalTokens": 158
}
Translation: 

Este Ford Explorer Platinum 2015 es la combinación perfecta de lujo y potencia. Está terminado en un elegante exterior plateado que destaca entre la multitud, y está alimentado por un motor V6 que proporci

index value 0 CarTranslation { VIN = 123, En = \n\nThis 2015 Ford Explorer Platinum is the perfect combination of luxury and power. It is finished in a sleek silver exterior that stands out from the crowd, and it is powered by a V6 engine that provides plenty of power. With only 10,000 miles, this Explorer is in ... VIN 123 En 

This 2015 Ford Explorer Platinum is the perfect combination of luxury and power. It is finished in a sleek silver exterior that stands out from the crowd, and it is powered by a V6 engine that provides plenty of power. With only 10,000 miles, this Explorer is in excellent condition and ready for its next owner. Inside, you'll find a luxurious interior with all the bells and whistles you could want. Enjoy features like leather seating, a power moonroof, and a premium sound system. All of this is available for the great price of only $16,500. Don't miss your chance to own this beautiful Ford Explorer! ES 

Este Ford Explorer Platinum 2015 es la combinación perfecta de lujo y potencia. Está terminado en un elegante exterior plateado que destaca entre la multitud, y está alimentado por un motor V6 que proporciona una gran cantidad de poder. Con solo 10,000 millas, este Explorer está en excelentes condiciones y listo para su próximo dueño. En el interior, encontrarás un lujoso interior con todos los adornos que podrías querer. Disfruta de características como asientos de cuero, un techo corredizo eléctrico y un sistema de sonido premium. Todo esto está disponible por el gran precio de solo $16,500. ¡No te pierdas la oportunidad 1 CarTranslation { VIN = 456, En = \n\nThis is your chance to own a piece of automotive history! This 2018 Ford Mustang Blue V8 Sports is a one-of-a-kind classic with only 10,000 miles on it. It has a powerful V8 engine, sleek exterior, and a comfortable interior. With a price tag of only $250,000, yo... VIN 456 En 

This is your chance to own a piece of automotive history! This 2018 Ford Mustang Blue V8 Sports is a one-of-a-kind classic with only 10,000 miles on it. It has a powerful V8 engine, sleek exterior, and a comfortable interior. With a price tag of only $250,000, you can take this classic car home and enjoy it for years to come. Don't miss out on this amazing opportunity to own a piece of automotive history! ES 

¡Esta es tu oportunidad de tener una pieza de la historia automotriz! Este Ford Mustang Azul V8 Sport de 2018 es único con solo 10,000 millas. Tiene un poderoso motor V8, un exterior elegante y un interior cómodo. Con un precio de solo $250,000, puedes llevar a casa este clásico automóvil y disfrutarlo durante años. ¡No te pierdas esta increíble oportunidad de tener una pieza de la historia automotriz! 2 CarTranslation { VIN = 789, En = \n\nThis is your chance to own a 2020 Ford Escape Red V6 Special with only 3000 miles for an amazing price of just $15000! This vehicle has been well maintained and is in excellent condition. It features a powerful V6 engine and special features including power windo... VIN 789 En 

This is your chance to own a 2020 Ford Escape Red V6 Special with only 3000 miles for an amazing price of just $15000! This vehicle has been well maintained and is in excellent condition. It features a powerful V6 engine and special features including power windows, power locks, air conditioning, and more. Don't miss out on this incredible opportunity to own a reliable and stylish car for a fraction of the cost. Come in today and take this Ford Escape Red V6 Special for a test drive! ES 

¡Esta es tu oportunidad de tener un Ford Escape Red V6 Especial 2020 con solo 3000 millas a un precio increíble de solo $15000! Este vehículo ha sido bien mantenido y está en excelentes condiciones. Cuenta con un potente motor V6 y características especiales como ventanas eléctricas, cerraduras eléctricas, aire acondicionado y más. No te pierdas esta increíble oportunidad de tener un auto confiable y elegante por una fracción del costo. ¡Ven hoy y haz una prueba de mane